### Contur data:

#### SM Background:

In [1]:
"""
From ATLAS_2022_I203774-Theort.yoda:
# xval	 xerr-	 xerr+	 yval	 yerr-	 yerr+	
3.680000e+02	1.300000e+01	1.300000e+01	1.279688e-02	3.288730e-03	2.684552e-04
4.005000e+02	1.950000e+01	1.950000e+01	9.729670e-03	2.573953e-03	2.325265e-04
4.490000e+02	2.900000e+01	2.900000e+01	6.077057e-03	1.592956e-03	2.043380e-04
5.135000e+02	3.550000e+01	3.550000e+01	3.098760e-03	8.270378e-04	1.097391e-04
5.910000e+02	4.200000e+01	4.200000e+01	1.362147e-03	3.663974e-04	5.330028e-05
6.765000e+02	4.350000e+01	4.350000e+01	5.439250e-04	1.298381e-04	8.712391e-06
7.780000e+02	5.800000e+01	5.800000e+01	1.954953e-04	4.865226e-05	1.035270e-05
1.418000e+03	5.820000e+02	5.820000e+02	9.935645e-06	2.420555e-06	1.003932e-06
"""

'\nFrom ATLAS_2022_I203774-Theort.yoda:\n# xval\t xerr-\t xerr+\t yval\t yerr-\t yerr+\t\n3.680000e+02\t1.300000e+01\t1.300000e+01\t1.279688e-02\t3.288730e-03\t2.684552e-04\n4.005000e+02\t1.950000e+01\t1.950000e+01\t9.729670e-03\t2.573953e-03\t2.325265e-04\n4.490000e+02\t2.900000e+01\t2.900000e+01\t6.077057e-03\t1.592956e-03\t2.043380e-04\n5.135000e+02\t3.550000e+01\t3.550000e+01\t3.098760e-03\t8.270378e-04\t1.097391e-04\n5.910000e+02\t4.200000e+01\t4.200000e+01\t1.362147e-03\t3.663974e-04\t5.330028e-05\n6.765000e+02\t4.350000e+01\t4.350000e+01\t5.439250e-04\t1.298381e-04\t8.712391e-06\n7.780000e+02\t5.800000e+01\t5.800000e+01\t1.954953e-04\t4.865226e-05\t1.035270e-05\n1.418000e+03\t5.820000e+02\t5.820000e+02\t9.935645e-06\t2.420555e-06\t1.003932e-06\n'

#### Signal

In [6]:
sig = [7.11288462e-07,1.72211154e-06,1.71594397e-06,1.18480085e-06, 6.48651071e-07, 2.41585517e-07, 1.15588276e-07, 7.22221478e-10]
uncertainties = [[6.21067227e-07,6.21067227e-07],
 [6.75766093e-07,6.75766093e-07],
 [4.89674276e-07,4.89674276e-07],
 [3.65062816e-07,3.65062816e-07],
 [2.43479023e-07,2.43479023e-07],
 [1.51971494e-07,1.51971494e-07],
 [8.61641516e-08,8.61641516e-08],
 [6.20525704e-09,6.20525704e-09]]

#### CLs values:

* SM BG = Data:
  * Reporting CLs 0.068387, pval 0.931613, p_sb 0.465807, p_b 0.500000, ts_s_b 0.007364, ts_b 0.000000

* SM BG = Theory:
  * Reporting CLs 0.008041, pval 0.991959, p_sb 0.027419, p_b 0.027641, ts_s_b 3.687001, ts_b 3.673539

* Expected (Data = SM BG):
  * Reporting CLs 0.003182, pval 0.996818, p_sb 0.498409, p_b 0.500000, ts_s_b 0.000016, ts_b 0.000000

## Recasting Data

#### SM Background